In [18]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

U_NP = [[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]]

def calculate_timbit(U, rho_0, rho, n_iters):
    """
    This function will return a timbit associated to the operator U and a state passed as an attribute.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        rho (numpy.tensor): A guess at the fixed point C[rho] = rho.
        n_iters (int): The number of iterations of C.

    Returns:
        (tuple(numpy.tensor)): The fixed point density matrices.
    """

    
    # Put your code here #
    dev = qml.device("default.mixed", wires=n_iters+1)
    @qml.qnode(dev)
    def circuit():
        qml.QubitDensityMatrix(rho, wires=n_iters)
        
        for i in range(n_iters):
            qml.QubitDensityMatrix(rho_0, wires=i)
            
        for ii in range(n_iters):    
            qml.QubitUnitary(U, wires=[ii,n_iters])
    
        return qml.density_matrix(wires=n_iters)    
    return circuit()
    
        


def apply_timbit_gate(U, rho_0, timbit):
    """
    Function that returns the output density matrix after applying a timbit gate to a state.
    The density matrix is the one associated with the first qubit.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        timbit (numpy.tensor): The timbit associated with the operator and the state.

    Returns:
        (tuple(numpy.tensor)): The output density matrices.
    """


    # Put your code here #
    
    dev = qml.device("default.mixed", wires=2)
    @qml.qnode(dev)
    def circuit():
        qml.QubitDensityMatrix(rho_0, wires=0)
        qml.QubitDensityMatrix(timbit, wires=1)
        qml.QubitUnitary(U, wires=[0,1])
    
        return qml.density_matrix(wires=0)
    return circuit()
    
    


def SAT(U_f, q, rho, n_bits):
    """A timbit-based algorithm used to guess if a Boolean function ever outputs 1.

    Args:
        U_f (numpy.tensor): A multi-qubit gate in matrix form.
        q (int): Number of times we apply the Timbit gate.
        rho (numpy.tensor): An initial guess at the fixed point C[rho] = rho.
        n_bits (int): The number of bits the Boolean function is defined on.

    Returns:
        numpy.tensor: The measurement probabilities on the last wire.
    """


    # Put your code here #
    
    dev = qml.device("default.mixed", wires=n_bits)
    @qml.qnode(dev)
    def rho0_circuit():
        for i in range(n_bits-1):
            qml.Hadamard(i)
        qml.QubitUnitary(U_f, wires=list(range(n_bits)))
    
        return qml.density_matrix(wires=n_bits-1)
    
    rho_0 = rho0_circuit()
    for i in range(q):
        timbit = calculate_timbit(U_NP, rho_0, rho, 3)
        rho_0 = apply_timbit_gate(U_NP, rho_0, timbit)
    
      
    return np.array([rho_0[0][0],rho_0[1][1]]).real.tolist()





In [20]:
I = np.eye(2)
X = qml.matrix(qml.PauliX(0))
U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
q = 2

'''
n_bits=4
dev = qml.device("default.mixed", wires=n_bits)
@qml.qnode(dev)
def rho0_circuit():
    for i in range(n_bits-1):
        qml.Hadamard(i)
    qml.QubitUnitary(U_f, wires=list(range(n_bits)))
    
    return qml.density_matrix(wires=n_bits-1)
    
rho_0 = rho0_circuit()
timbit=calculate_timbit(U_NP, rho_0, rho, 3)

dev1 = qml.device("default.mixed", wires=1+n_bits)
@qml.qnode(dev1)
def circuit():
    qml.QubitDensityMatrix(timbit, wires=n_bits)
    for i in range(n_bits-1):
        qml.Hadamard(i)
    qml.QubitUnitary(U_f, wires=list(range(n_bits)))
    
    qml.QubitUnitary(U_NP, wires=[n_bits-1, n_bits])
    
    
    return qml.density_matrix(wires=n_bits-1)
rhof= circuit()

print(rhof)
'''
str(SAT(U_f, q, rho,4))

'[0.6582031249999952, 0.34179687499999756]'

In [17]:
rho_0 = np.array([[0.875+0.j, 0.   +0.j], [0.   +0.j, 0.125+0.j]])
timbit = np.array([[0.875+0.j, 0.   +0.j], [0.   +0.j, 0.125+0.j]])
rho_1 = np.array([[0.78125+0.j, 0.     +0.j],
       [0.     +0.j, 0.21875+0.j]])
dev = qml.device("default.mixed", wires=2)
@qml.qnode(dev)
def circuitq():
    qml.QubitDensityMatrix(rho_1, wires=0)
    qml.QubitDensityMatrix(timbit, wires=1)
    qml.QubitUnitary(U_NP, wires=[0,1])
    
    return qml.density_matrix(wires=0)

rho2=circuitq()
rho2

array([[0.7109375+0.j, 0.       +0.j],
       [0.       +0.j, 0.2890625+0.j]])

In [16]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    I = np.eye(2)
    X = qml.matrix(qml.PauliX(0))

    U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    q = json.loads(test_case_input)
    output = list(SAT(U_f, q, rho,4))

    return str(output)

def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
    rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    assert np.allclose(
        solution_output, expected_output, atol=0.01
    ), "Your NP-solving timbit computer isn't quite right yet!"


test_cases = [['1', '[0.78125, 0.21875]'], ['2', '[0.65820312, 0.34179687]']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1'...
Correct!
Running test case 1 with input '2'...


AssertionError: Your NP-solving timbit computer isn't quite right yet!

In [15]:
np.matmul(np.array(U_NP),np.array(U_NP))

tensor([[1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]], requires_grad=True)

In [2]:
a*b

tensor([[1, 2],
        [3, 4]], requires_grad=True)

In [28]:
(2-1)*2+1

3